<a href="https://colab.research.google.com/github/gevargas/bigdata-management/blob/master/mrjobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Ce texte est au format code
```

# Example 3: counting the number of lines of a file (mrjob python library)

* Install mrjob

In [ ]:
!pip install mrjob

     |████████████████████████████████| 439 kB 5.1 MB/s 


* Copy mrjob examples

In [ ]:
!cp -r /usr/local/lib/python3.7/dist-packages/mrjob/examples .

* Count frequent words containing the letter 'u'

In [ ]:
!python \
     /content/examples/mr_words_containing_u_freq_count.py -r local \
     /content/examples/docs-to-classify/twinkletoes-milne.txt

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/mr_words_containing_u_freq_count.root.20220321.140921.290809
Running step 1 of 1...
job output is in /tmp/mr_words_containing_u_freq_count.root.20220321.140921.290809/output
Streaming final output from /tmp/mr_words_containing_u_freq_count.root.20220321.140921.290809/output...
"through"	1
"sun"	2
Removing temp directory /tmp/mr_words_containing_u_freq_count.root.20220321.140921.290809...


# Ex: Which files are we reading input from?

In [ ]:
%%writefile mr_count_lines_by_file.py

"""Which files are we reading input from?"""
from mrjob.compat import jobconf_from_env
from mrjob.job import MRJob

class MRCountLinesByFile(MRJob):

    def mapper(self, _, line):
        yield jobconf_from_env('mapreduce.map.input.file'), 1

    def reducer(self, path, ones):
        yield path, sum(ones)

if __name__ == '__main__':
    MRCountLinesByFile.run()

Writing mr_count_lines_by_file.py


In [ ]:
input = "examples/docs-to-classify/*"
mrjob = "mr_count_lines_by_file.py"

!python $mrjob $input

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/mr_count_lines_by_file.root.20220321.142703.025987
Running step 1 of 1...
job output is in /tmp/mr_count_lines_by_file.root.20220321.142703.025987/output
Streaming final output from /tmp/mr_count_lines_by_file.root.20220321.142703.025987/output...
"file://examples/docs-to-classify/corner_of_the_street-milne-not_whitman.txt"	8
"file://examples/docs-to-classify/happiness-milne.txt"	16
"file://examples/docs-to-classify/in_cabind_ships_at_sea-whitman.txt"	31
"file://examples/docs-to-classify/lines_and_squares-milne-animals.txt"	9
"file://examples/docs-to-classify/ones_self_i_sing-whitman.txt"	11
"file://examples/docs-to-classify/puppy_and_i-milne-animals.txt"	35
"file://examples/docs-to-classify/the_christening-milne-animals.txt"	14
"file://examples/docs-to-classify/the_four_friends-milne-animals.txt"	20
"file://examples/docs-to-classify/to_a_historian-whitman.txt"	11
"f

# Example 4: grep

In [ ]:
!cat /content/examples/docs-to-classify/chants_democratic-whitman-america.txt

STARTING FROM PAUMANOK.
1.

Starting from fish-shape Paumanok,[1] where I was born,
Well-begotten, and raised by a perfect mother;
After roaming many lands—lover of populous pavements;
Dweller in Mannahatta,[2] city of ships, my city,—or on southern savannas;
Or a soldier camped, or carrying my knapsack and gun—or a miner in
        California;
Or rude in my home in Dakotah's woods, my diet meat, my drink from the
        spring;
Or withdrawn to muse and meditate in some deep recess,
Far from the clank of crowds, intervals passing, rapt and happy;
Aware of the fresh free giver, the flowing Missouri—aware of mighty
        Niagara
Aware of the buffalo herds, grazing the plains—the hirsute and strong-
        breasted bull;
Of earths, rocks, fifth-month flowers, experienced—stars, rain, snow, my
        amaze;
Having studied the mocking-bird's tones, and the mountain hawk's,
And heard at dusk the unrivalled one, the hermit thrush, from the
        swamp-cedars,
Solitary, singing in the W

In [ ]:
!python \
     /content/examples/mr_grep.py  -r local \
     /content/examples/docs-to-classify/chants_democratic-whitman-america.txt -e 'Or '

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/mr_grep.root.20220321.151956.606456
Running step 1 of 1...
job output is in /tmp/mr_grep.root.20220321.151956.606456/output
Streaming final output from /tmp/mr_grep.root.20220321.151956.606456/output...
Or if there is, I say it is just as important to you, to the land, or to
Or a soldier camped, or carrying my knapsack and gun—or a miner in
Or rude in my home in Dakotah's woods, my diet meat, my drink from the
Or withdrawn to muse and meditate in some deep recess,
Removing temp directory /tmp/mr_grep.root.20220321.151956.606456...


How is this program implemented?

In [ ]:
"""Return all input lines matching the given expression."""
from mrjob.job import MRJob
from mrjob.util import cmd_line


class MRGrepJob(MRJob):

    def configure_args(self):
        super(MRGrepJob, self).configure_args()

        self.add_passthru_arg(
            '-e', '--expression',
            required=True,
            help=('Expression to search for. Required.'))

    def mapper_cmd(self):
        # grep will return exit status 1 if no matching lines are found
        return cmd_line([
            'sh', '-c',
            'grep -e %s || RC=$?; if [ $RC -ne 1 ]; then (exit $RC); fi' % (
                cmd_line([self.options.expression]))])


if __name__ == '__main__':
    MRGrepJob.run()

# Example 5: Identifying the most use word

In [ ]:
!cat /content/examples/docs-to-classify/american_feuillage-whitman-america.txt

AMERICA always!
Always our own feuillage!
Always Florida's green peninsula! Always the priceless delta of Louisiana!
Always the cotton-fields of Alabama and Texas!
Always California's golden hills and hollows—and the silver mountains of
New Mexico! Always soft-breathed Cuba!
Always the vast slope drained by the Southern Sea—inseparable with the
        slopes drained by the Eastern and Western Seas!
The area the eighty-third year of these States[1]—the three and a half
        millions of square miles;
The eighteen thousand miles of sea-coast and bay-coast on the main—the
        thirty thousand miles of river navigation,
The seven millions of distinct families, and the same number of dwellings—
Always these, and more, branching forth into numberless branches;
Always the free range and diversity! Always the continent of Democracy!
Always the prairies, pastures, forests, vast cities, travellers, Canada,
        the snows;
Always these compact lands—lands tied at the hips with the belt s

In [ ]:
!python \
     /content/examples/mr_most_used_word.py  -r local \
     /content/examples/docs-to-classify/american_feuillage-whitman-america.txt

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/mr_most_used_word.root.20220321.153015.551646
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/mr_most_used_word.root.20220321.153015.551646/output
Streaming final output from /tmp/mr_most_used_word.root.20220321.153015.551646/output...
"always"
Removing temp directory /tmp/mr_most_used_word.root.20220321.153015.551646...


How is this program implemented?

In [ ]:
"""Determine the most used word in the input, ignoring common "stop" words.
Shows how to do a multi-step job, and how to load a support file
from the same directory.
"""
import re

from mrjob.job import MRJob
from mrjob.protocol import JSONValueProtocol
from mrjob.step import MRStep

WORD_RE = re.compile(r"[\w']+")


class MRMostUsedWord(MRJob):
    FILES = ['stop_words.txt']

    OUTPUT_PROTOCOL = JSONValueProtocol

    def configure_args(self):
        super(MRMostUsedWord, self).configure_args()

        # allow for alternate stop words file
        self.add_file_arg(
            '--stop-words-file',
            dest='stop_words_file',
            default=None,
            help='alternate stop words file. lowercase words, one per line',
        )

    def mapper_init(self):
        stop_words_path = self.options.stop_words_file or 'stop_words.txt'

        with open(stop_words_path) as f:
            self.stop_words = set(line.strip() for line in f)

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            word = word.lower()
            if word not in self.stop_words:
                yield (word, 1)

    def combiner_count_words(self, word, counts):
        # sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        try:
            yield max(word_count_pairs)
        except ValueError:
            pass

    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]


if __name__ == '__main__':
    MRMostUsedWord.run()

# Example 6: Statistics
*For any word that appears in a document, compute stats about which
words come next (including percentage) (mrjob python library)*

In [ ]:
!cat /content/examples/docs-to-classify/twinkletoes-milne.txt

When the sun
Shines through the leaves of the apple-tree,
When the sun
Makes shadows of the leaves of the apple-tree,
Then I pass
On the grass
From one leaf to another
From one leaf to its brother
Tip-toe, tip-toe!
Here I go!


Twinkletoes, A. A. Milne https://allpoetry.com/poem/8518961-Twinkletoes-by-A.A.-Milne: A. A. (Alan Alexander) Milne (1882-1956), famous for his stories about Winnie the Pooh and Christopher Robin, Tigger, Piglet and the rest.

In [ ]:
!python \
     /content/examples/mr_next_word_stats.py -r local \
     /content/examples/docs-to-classify/twinkletoes-milne.txt

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/mr_next_word_stats.root.20220321.144849.416528
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/mr_next_word_stats.root.20220321.144849.416528/output
Streaming final output from /tmp/mr_next_word_stats.root.20220321.144849.416528/output...
["leaves", "of"]	[2, 2, 100.0]
["makes", "shadows"]	[1, 1, 100.0]
["of", "the"]	[3, 3, 100.0]
["on", "the"]	[1, 1, 100.0]
["one", "leaf"]	[2, 2, 100.0]
["shadows", "of"]	[1, 1, 100.0]
["shines", "through"]	[1, 1, 100.0]
["the", "apple"]	[7, 2, 28.571428571428573]
["the", "grass"]	[7, 1, 14.285714285714286]
["the", "leaves"]	[7, 2, 28.571428571428573]
["the", "sun"]	[7, 2, 28.571428571428573]
["then", "i"]	[1, 1, 100.0]
["through", "the"]	[1, 1, 100.0]
["tip", "toe"]	[2, 2, 100.0]
["to", "another"]	[2, 1, 50.0]
["to", "its"]	[2, 1, 50.0]
["toe", "tip"]	[1, 1, 100.0]
["when", "the"]	[2, 2, 100.0]
["apple", "tree"]	[

How is this program implemented?

In [ ]:
"""For any word that appears in a document, compute stats about which
words come next (including percentage).
This is meant as a simple demonstration of why SORT_VALUES is useful.
"""
from mrjob.job import MRJob
from mrjob.step import MRStep
import re


WORD_RE = re.compile(r"[\w']+")


class MRNextWordStats(MRJob):

    SORT_VALUES = True

    def steps(self):
        return [MRStep(mapper=self.m_find_words,
                       combiner=self.c_combine_counts,
                       reducer=self.r_sum_counts),
                MRStep(reducer=self.r_compute_stats)]

    def m_find_words(self, _, line):
        """Tokenize lines, and look for pairs of adjacent words.
        Yield (prev_word, word), 1 and (prev_word, '*'), 1 for each pair
        """
        prev_word = None

        for word in WORD_RE.findall(line):
            word = word.lower()

            if prev_word is not None:
                # total up the number of times prev_word appears
                # and the number of times next_word appears after it
                yield (prev_word, '*'), 1
                yield (prev_word, word), 1

            prev_word = word

    def c_combine_counts(self, key, counts):
        """Sum up all those 1s before passing data off to the reducer"""
        yield key, sum(counts)

    def r_sum_counts(self, key, counts):
        """Compute the number of times each pair of words appears, and the
        number of times the first word in a pair appears, and send it to
        a reducer that keys on the first word in the pair.
        """
        count = sum(counts)

        prev_word, word = key

        if word == '*':
            # we want total to arrive at r_compute_stats first, so
            # prefix it with "A", which comes before "B"
            yield prev_word, ('A: total', count)
        else:
            yield prev_word, ('B: stats', (word, count))

    def r_compute_stats(self, prev_word, value):
        """For each pair of words, compute how many times it appears,
        how many times the first word appears in a pair, and the percentage
        of time the second word follows the first.
        This relies on values appearing in sorted order; we need the total
        number of times the first word appears before we can compute the
        percentage for each second word.
        """
        total = None

        for value_type, data in value:
            if value_type == 'A: total':
                total = data
            else:
                assert value_type == 'B: stats'
                word, count = data
                # A comes before B, so total should already be set
                percent = 100.0 * count / total
                yield (prev_word, word), (total, count, percent)


if __name__ == '__main__':
    MRNextWordStats.run()